# Elektrogeräte und Kunststoffe

In [18]:
import pandas as pd

### Create dataframes from csv files

In [19]:
path1 = 'database1.csv'
path2 = 'database2.csv'
path3 = 'database3.csv'

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

## Add new column "Anlage" to uniquely identify the site within one column + merge the two dataframes

In [20]:
# To show all columns
pd.set_option('display.max_columns', None)
df1 = df1.fillna('')
df2 = df2.fillna('')

# Eine weitere Spalte erstellen (ANLAGE), die den Namen des Betriebs + dessen Ort enthält
df1["ANLAGE"]=df1["BETRIEB_NAME"] + " " + df1["BETRIEB_ORT"]
df2["ANLAGE"]=df2["BETRIEB_NAME"] + " " + df2["BETRIEB_ORT"]


# Move new Column (ANLAGE) to the start of the dataframe
df1.insert(0, 'ANLAGE', df1.pop('ANLAGE'))
df2.insert(0, 'ANLAGE', df2.pop('ANLAGE'))


# What columns are in df2 that are not in df1
columns_to_append = [col for col in df2.columns if col not in df1.columns]

# Append the selected columns from df2 to df1
df1 = pd.concat([df1, df2[columns_to_append]], axis=1)

print("DataFrame from database1.csv:\n")


DataFrame from database1.csv:


C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_8496\3803874098.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1["ANLAGE"]=df1["BETRIEB_NAME"] + " " + df1["BETRIEB_ORT"]
C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_8496\3803874098.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df1.insert(0, 'ANLAGE', df1.pop('ANLAGE'))


## Change reported values from the company "Noris-Arbeit gGmbH" to kg

In [21]:
def divide_by_1000(value):
    if isinstance(value, (int, float)):
        return value / 1000
    return value

df1.loc[df1['BETRIEB_NAME'] == 'Noris-Arbeit gGmbH', df1.columns != 'BETRIEB_NAME'] = df1.loc[df1['BETRIEB_NAME'] == 'Noris-Arbeit gGmbH', df1.columns != 'BETRIEB_NAME'].applymap(divide_by_1000)

## Definition of functions

In [22]:
def sum_float_values(column):
    total = 0.0
    for value in column:
        if isinstance(value, (float, int)):
            total += value
    return total

In [23]:
# Defining function that checks whether total values or percentage values are missing (in case one of them has an entry)
def determine_missing_vals(dataframe, total_value_col, perc_value_col):

    missing_values = {"total_missing": 0, "perc_missing": 0}
    
    # Check if the specified columns exist in the DataFrame
    if total_value_col not in dataframe.columns or perc_value_col not in dataframe.columns:
        raise ValueError("One or both of the specified columns do not exist in the DataFrame.")

    # Iterate through the rows and perform the comparison
    for index, row in dataframe.iterrows():
        if row[total_value_col] == '' and row[perc_value_col] != '':
            missing_values["total_missing"] += 1
        elif row[total_value_col] != '' and row[perc_value_col] == '':
            missing_values["perc_missing"] += 1

    return missing_values

## Nach einzelnen Anlagen Filtern

In [24]:
# Nach einzelnen Anlagen (Name Betrieb + Ort des Betriebs) schauen. Falls alle Anlagen betrachtet werden sollen, das Feld leer lassen.
# Es kann auch nach Betrieben gesucht werden (dann werden alle Anlagen des Betrieb mit einbezogen)
anlage = ""

if len(anlage) != 0:
    df1 = df1[df1['ANLAGE'].str.lower().str.contains(anlage.lower())]
    
print("Anzahl der Observationen für die gegebene Anlange: ", len(df1))

Anzahl der Observationen für die gegebene Anlange:  363


## Gerätekategorie bestimmen

In [25]:
category = "KG1"
inputCategory = category + "_INPUT"

### Gesamter Input in EBA (für Gerätekategorie und Anlage)

In [26]:
# Total Input of category
KG_Input_Sum = sum_float_values(df1[inputCategory])
print(KG_Input_Sum)

134992.0


### Output (Total und KS) der EBA (für Gerätekategorie und Anlage)

In [27]:
# Computation of overall output of EBA
column = category + "_OUT_GES_MASSE"
ges_out_eba = sum_float_values(df1[column])
print("Gesamter Output der EBA für Kategorie", category, ": ", ges_out_eba)

# Computation of overall KS output of EBA
column = category + "_OUT_KS_MASSE"
ks_out_eba = sum_float_values(df1[column])
print("Kunststoff Output aus der EBA für ", category, ": ", ks_out_eba)

Gesamter Output der EBA für Kategorie KG1 :  122655.0
Kunststoff Output aus der EBA für  KG1 :  32597.0


### Input in VWA (für Gerätekategorie und Anlage)

In [28]:
# Computation of overall Input in VWA
column = category + "_TRT_GES"
ges_in_vwa = sum_float_values(df1[column])
print("Gesamter Input für VWA für Kategorie", category + ":", ges_in_vwa)

# Computation of KS Input in VWA
column = category + "_TRT_KS"
ks_in_vwa = sum_float_values(df1[column])
print("Kunststoff Input in VWA für Kategorie", category + ":", ks_in_vwa)

# Computation of overall Recycling Input in VWA
column = category + "_TRT_RCY"
rec_in_vwa = sum_float_values(df1[column])
print("Recycling Input in VWA für Kategorie", category + ":", rec_in_vwa)

# Computation of overall KS Recycling Input in VWA
column = category + "_TRT_RCY_KS"
rec_ks_in_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Input in VWA für Kategorie", category + ":", rec_ks_in_vwa)

# Computation of overall Sonstige Verwertung Input in VWA
column = category + "_TRT_RCV_OTH"
sonVW_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Input in VWA für Kategorie", category + ":", sonVW_in_vwa)

# Computation of overall KS sonstige Verwertung Input in VWA
column = category + "_TRT_RCV_OTH_KS"
sonVW_ks_in_vwa = sum_float_values(df1[column])
print("Sonstige Verwertung Kunststoff Input in VWA für Kategorie", category + ":", sonVW_ks_in_vwa)

Gesamter Input für VWA für Kategorie KG1: 74571.0
Kunststoff Input in VWA für Kategorie KG1: 32498.0
Recycling Input in VWA für Kategorie KG1: 63664.0
Recycling Kunststoff Input in VWA für Kategorie KG1: 19658.0
Sonstige Verwertung Input in VWA für Kategorie KG1: 35282.0
Sonstige Verwertung Kunststoff Input in VWA für Kategorie KG1: 8246.0


### Outputs der VWA (für Gerätekategorie und Anlage)

In [29]:
# Computation of overall Recycling Output of VWA
column = category + "_OUT_RCY_GES"
ges_rec_out_vwa = sum_float_values(df1[column])
print("Gesamter Recycling Output der VWA für Kategorie", category + ":", ges_rec_out_vwa)

# Computation of overall KS Recycling Output of VWA
column = category + "_OUT_RCY_KS"
ks_rec_out_vwa = sum_float_values(df1[column])
print("Recycling Kunststoff Output der VWA für Kategorie", category + ":", ks_rec_out_vwa)

Gesamter Recycling Output der VWA für Kategorie KG1: 60608.0
Recycling Kunststoff Output der VWA für Kategorie KG1: 14781.0


## Assessment of credibility of reported numbers (Input und Output der EBA)

In [30]:
# Interval for ratio of Recycling-Output EBA/total Input EBA
rec_min = 0.1
rec_max = 0.6

# Relevant columns (Gesamt-Output of EBA and KS-Output of EBA)
eba_total_out = category + "_OUT_GES_MASSE"
column = category + "_OUT_KS_MASSE"

# Get rid of rows that did not specify value for the respective input category
df1 = df1[(pd.notna(df1[inputCategory])) & (~df1[inputCategory].astype(str).str.isdigit())]
df1 = df1[df1[inputCategory].astype(str).str.contains('\d', na=False) & (df1[inputCategory] != 0)]

# Reset Index
df1 = df1.reset_index(drop=True)

# Convert the specified columns to numeric, coercing errors to NaN
df1[column] = pd.to_numeric(df1[column], errors='coerce')
df1[eba_total_out] = pd.to_numeric(df1[eba_total_out], errors='coerce')


# How much is the total output of EBA compared to its input
df1["eba_ges_input_vs_output"] = df1[eba_total_out]/df1[inputCategory]

# Zero Mask and NaN Mask to determine missing or "0" values 
zero_mask = df1[eba_total_out] == 0 
nan_mask = pd.isna(df1[eba_total_out])

# Print number of empty or "0" entries for Total Output of the EBA
print(f"Anzahl von 0 Einträgen für totalen Output der EBA: {zero_mask.sum()}")
print(f"Anzahl von keinen Einträgen für totalen Output der EBA: {nan_mask.sum()}")

# Calculation for Recycling Output of EBA
eba_rec_no_entry_count = df1[column].isna().sum()
eba_rec_zero_entry_count = (df1[column] == 0).sum()

# Parse entries in inputCategory to numbers
df1[inputCategory] = pd.to_numeric(df1[inputCategory], errors='coerce')

# Calculate share of KS Output from total Input in EBA
df1["share_KS_Out_EBA_tot_input"] =  df1[column]/df1[inputCategory]

# interval for ratio total_output_eba/total_input_eba
minout = 0.8
maxout = 1.0

# How much values of the ratio between total input and total output of eba lie in the interval
cred_total_output_count = df1[(df1['eba_ges_input_vs_output'] >= minout) & (df1['eba_ges_input_vs_output'] <= maxout)].shape[0]

# Output of the EBA exceeds its Input 
more_than_100 = df1[(df1['eba_ges_input_vs_output'] > 1)].shape[0]
# Output of EBA is less than 95% of its Input
less_than_95 = df1[(df1['eba_ges_input_vs_output'] < 0.95)].shape[0]

# Statements about total Input to total Output Ratio of the EBA
print(f"#Betriebe mit plausiblen Gesamt-Outputmengen der EBA zwischen {minout*100}% und {maxout*100}%: {cred_total_output_count}")
print(f"\t #Betriebe mit Quote > 1: {more_than_100}")
print(f"\t #Betriebe mit Quote < 0.95: {less_than_95}")


cat_input_count = len(df1)

eba_recshare_no_entry_count = df1['share_KS_Out_EBA_tot_input'].isna().sum()
eba_recshare_zero_entry_count = (df1['share_KS_Out_EBA_tot_input'] == 0).sum()

#
print(f"Anzahl der Anlagen die einen positiven Eintrag für die Inputkategorie haben: {cat_input_count}")
print(f"\t Davon Anzahl der Anlagen die 0 als Recycling Output für KS der EBA angeben: {eba_rec_zero_entry_count}")
print(f"\t Davon Anzahl der Anlagen die KEINE Angaben für den KS Output der EBA angeben: {eba_rec_no_entry_count}")

num_cred_recshare = df1[(df1['share_KS_Out_EBA_tot_input'] >= rec_min) & (df1['share_KS_Out_EBA_tot_input'] <= rec_max)].shape[0]

print(f"\nAnzahl der Anlagen die glaubhafe Recyclingquote (zwischen {rec_min*100}% und {rec_max*100}%) für KS in der EBA angeben: {num_cred_recshare}")



Anzahl von 0 Einträgen für totalen Output der EBA: 0
Anzahl von keinen Einträgen für totalen Output der EBA: 5
#Betriebe mit plausiblen Gesamt-Outputmengen der EBA zwischen 80.0% und 100.0%: 17
	 #Betriebe mit Quote > 1: 4
	 #Betriebe mit Quote < 0.95: 2
Anzahl der Anlagen die einen positiven Eintrag für die Inputkategorie haben: 27
	 Davon Anzahl der Anlagen die 0 als Recycling Output für KS der EBA angeben: 3
	 Davon Anzahl der Anlagen die KEINE Angaben für den KS Output der EBA angeben: 5

Anzahl der Anlagen die glaubhafe Recyclingquote (zwischen 10.0% und 60.0%) für KS in der EBA angeben: 18


## Assessment of credibility of reported numbers (Output EBA und Input VWA)

In [31]:
outeba_invwa_min = 0.8
outeba_invwa_max = 1.0

eba_output = category + "_OUT_KS_MASSE"
vwa_input = category + "_TRT_KS"
vwa_input_rec = category + "_TRT_RCY_KS"
vwa_input_sonstVW = category + "_TRT_RCV_OTH_KS"

vwa_total_in = category + "_TRT_GES"

df1[vwa_input] = pd.to_numeric(df1[vwa_input], errors='coerce')
df1[vwa_input_rec] = pd.to_numeric(df1[vwa_input_rec], errors='coerce')
df1[vwa_input_sonstVW] = pd.to_numeric(df1[vwa_input_sonstVW], errors='coerce')
df1[vwa_total_in] = pd.to_numeric(df1[vwa_total_in], errors='coerce')

df1["eba_total_out_vwa_in"] = df1[vwa_total_in]/df1[eba_total_out]
print(df1["eba_total_out_vwa_in"])

zero_mask = df1[vwa_total_in] == 0 
nan_mask = pd.isna(df1[vwa_total_in])


print(f"Anzahl von 0 Einträgen für totalen Input der VWA: {zero_mask.sum()}")
print(f"Anzahl von keinen Einträgen für totalen Input der VWA: {nan_mask.sum()}")



df1["outeba_invwa_quote"] = df1[vwa_input]/df1[eba_output]


minin = .8
minout = 1.0

num_cred_vwa_input_total = df1[(df1['eba_total_out_vwa_in'] >= minin) & (df1['eba_total_out_vwa_in'] <= minout)].shape[0]

print(f"Anzahl glaubhafter Eintragungen der totalen Werte in VWA (zwischen {minin*100}% und {minout*100}%) in Anbetracht des totalen Outputs der EBA: {num_cred_vwa_input_total}")


no_input_vwa_count_overall = df1[(df1[vwa_input] != 0) & (~pd.isna(df1[vwa_input]))].shape[0]

no_input_vwa_count = df1[(df1[eba_output] > 0) & pd.isna(df1[vwa_input])].shape[0]

num_cred_vwa_input = df1[(df1['outeba_invwa_quote'] >= outeba_invwa_min) & (df1['outeba_invwa_quote'] <= outeba_invwa_max)].shape[0]

print(f"Anzahl der positiven Eintragungen für den Input von KS in die VWA: {no_input_vwa_count_overall}")
print(f"Anzahl der leeren Inputwerte für KS in die VWA, wo Output der EBA angegeben ist: {no_input_vwa_count}")
print(f"Anzahl glaubhafter Eintragungen der KS in VWA (zwischen {outeba_invwa_min*100}% und {outeba_invwa_max*100}%) in Anbetracht des Outputs der EBA: {num_cred_vwa_input}")

df1["sum_rec_input_vwa"] = df1[vwa_input_rec] + df1[vwa_input_sonstVW]


count_sum_greater_overall= (df1['sum_rec_input_vwa'] > df1[vwa_input]).sum()

print(f"Anzahl der Eintragungen wo Summe des Recycling und der sonstigen Verwertung der KS größer ist als gesamter Kunststoff-Input: {count_sum_greater_overall}")
print()

# Anteil des Recycling KS am Gesamt KS Input in VWA
df1["vwa_input_rec_quote"] = df1[vwa_input_rec]/df1[vwa_input]

# Setzen der min- und max-Werte für Quote der glaubhaften Werte des Recycling KS Inputs in die VWA
rec_invwa_quote_min = 0.2
rec_invwa_quote_max = 1.0


num_cred_vwa_input_recquote = df1[(df1['vwa_input_rec_quote'] >= rec_invwa_quote_min) & (df1['vwa_input_rec_quote'] <= rec_invwa_quote_max)].shape[0]

print(f"Anzahl der Eintragungen in VWA wo Anteil des Recycling Input von KS am Gesamtinput von KS glaubhaft ist: {num_cred_vwa_input_recquote}")




0     1.000000
1     1.000000
2     1.000000
3          NaN
4          NaN
5     0.258935
6     1.000000
7     0.258985
8     1.000000
9     1.000000
10    0.258905
11    1.000000
12    1.000000
13    0.260047
14    0.260022
15         NaN
16    0.985101
17    1.000000
18    1.000000
19         NaN
20    1.000000
21    1.000000
22    1.000000
23    0.995506
24    1.000000
25         NaN
26    0.763889
Name: eba_total_out_vwa_in, dtype: float64
Anzahl von 0 Einträgen für totalen Input der VWA: 0
Anzahl von keinen Einträgen für totalen Input der VWA: 5
Anzahl glaubhafter Eintragungen der totalen Werte in VWA (zwischen 80.0% und 100.0%) in Anbetracht des totalen Outputs der EBA: 16
Anzahl der positiven Eintragungen für den Input von KS in die VWA: 19
Anzahl der leeren Inputwerte für KS in die VWA, wo Output der EBA angegeben ist: 0
Anzahl glaubhafter Eintragungen der KS in VWA (zwischen 80.0% und 100.0%) in Anbetracht des Outputs der EBA: 19
Anzahl der Eintragungen wo Summe des Recycling 

## Assessment of Credibility of reported Numbers (Input und Output der VWA) 

In [32]:
quote_min = 0.5
quote_max = 1.0

vwa_output_rec = category + "_OUT_RCY_KS"

df1[vwa_output_rec] = pd.to_numeric(df1[vwa_output_rec], errors='coerce')

# Quote Input/Output der KS Recyling der VWA
# df1[vwa_input_rec_input_output_quote] = df1[vwa_output_rec]/df1[vwa_input_rec]

zero_mask = df1[vwa_input_rec] == 0 
nan_mask = pd.isna(df1[vwa_input_rec])
df1["vwa_input_rec_input_output_quote"] = 0
for ind in df1.index:
    if not (zero_mask[ind] or nan_mask[ind]): 
        
        df1["vwa_input_rec_input_output_quote"][ind] = df1[vwa_output_rec][ind]/df1[vwa_input_rec][ind]

        
num_cred_vwa_input_recquote = df1[(df1['vwa_input_rec_input_output_quote'] >= quote_min) & (df1['vwa_input_rec_quote'] <= quote_max)].shape[0]
print(df1[vwa_input_rec])
print(df1[vwa_output_rec])
print(df1["vwa_input_rec_input_output_quote"])

print()
print(num_cred_vwa_input_recquote)

0        0.0
1      125.0
2     2582.0
3        NaN
4        NaN
5     1602.0
6        NaN
7     2873.0
8       12.0
9       38.0
10     880.0
11       1.0
12    1209.0
13    1519.0
14    3914.0
15       NaN
16    3286.0
17       4.0
18       0.0
19       NaN
20     104.0
21      19.0
22      55.0
23     512.0
24     923.0
25       NaN
26       0.0
Name: KG1_TRT_RCY_KS, dtype: float64
0        0.0
1       81.0
2     1756.0
3        NaN
4        NaN
5     1155.0
6        NaN
7     2072.0
8        4.0
9       17.0
10     634.0
11       0.0
12    1141.0
13    1063.0
14    2740.0
15       NaN
16    2370.0
17       4.0
18       0.0
19       NaN
20     215.0
21      39.0
22      55.0
23     512.0
24     923.0
25       NaN
26       0.0
Name: KG1_OUT_RCY_KS, dtype: float64
0     0.000000
1     0.648000
2     0.680093
3     0.000000
4     0.000000
5     0.720974
6     0.000000
7     0.721197
8     0.333333
9     0.447368
10    0.720455
11    0.000000
12    0.943755
13    0.699803
14    0.700051

C:\Users\Felix Medicus\AppData\Local\Temp\ipykernel_8496\3968547574.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["vwa_input_rec_input_output_quote"][ind] = df1[vwa_output_rec][ind]/df1[vwa_input_rec][ind]
